# Biostat M280 Homework 2

**Due May 11 @ 11:59PM**

## Q1. Nonnegative Matrix Factorization

Nonnegative matrix factorization (NNMF) was introduced by [Lee and Seung (1999)](https://www.nature.com/articles/44565) as an analog of principal components and vector quantization with applications in data compression and clustering. In this homework we consider algorithms for fitting NNMF and (optionally) high performance computing using graphical processing units (GPUs).

<img src="./nnmf.png" width="500" align="center"/>

In mathematical terms, one approximates a data matrix $\mathbf{X} \in \mathbb{R}^{m \times n}$ with nonnegative entries $x_{ij}$ by a product of two low-rank matrices $\mathbf{V} \in \mathbb{R}^{m \times r}$ and $\mathbf{W} \in \mathbb{R}^{r \times n}$ with nonnegative entries $v_{ik}$ and $w_{kj}$. Consider minimization of the squared Frobenius norm
$$
	L(\mathbf{V}, \mathbf{W}) = \|\mathbf{X} - \mathbf{V} \mathbf{W}\|_{\text{F}}^2 = \sum_i \sum_j \left(x_{ij} - \sum_k v_{ik} w_{kj} \right)^2, \quad v_{ik} \ge 0, w_{kj} \ge 0,
$$
which should lead to a good factorization. Later in the course we will learn how to derive a majorization-minimization (MM) algorithm with iterative updates
$$
	v_{ik}^{(t+1)} = v_{ik}^{(t)} \frac{\sum_j x_{ij} w_{kj}^{(t)}}{\sum_j b_{ij}^{(t)} w_{kj}^{(t)}}, \quad \text{where } b_{ij}^{(t)} = \sum_k v_{ik}^{(t)} w_{kj}^{(t)},
$$
$$
	w_{kj}^{(t+1)} = w_{kj}^{(t)} \frac{\sum_i x_{ij} v_{ik}^{(t+1)}}{\sum_i b_{ij}^{(t+1/2)} v_{ik}^{(t+1)}}, \quad \text{where } b_{ij}^{(t+1/2)} = \sum_k v_{ik}^{(t+1)} w_{kj}^{(t)}
$$
that drive the objective $L^{(t)} = L(\mathbf{V}^{(t)}, \mathbf{W}^{(t)})$ downhill. Superscript $t$ indicates iteration number. Efficiency (both speed and memory) will be the most important criterion when grading this problem.


1. Implement the algorithm with arguments: $\mathbf{X}$ (data, each row is a vectorized image), rank $r$, convergence tolerance, and optional starting point.
```julia
function nnmf(
    X::Matrix, 
    r::Int;
    maxiter::Int=1000, 
    tol::eltype(X)=1e-4,
    V::Matrix{eltype(X)}=rand(size(X, 1), r),
    W::Matrix{eltype(X)}=rand(r, size(X, 2))
    )
    # implementation
    # Output
    return V, W
end
```

0. Database 1 from the [MIT Center for Biological and Computational Learning (CBCL)](http://cbcl.mit.edu) reduces to a matrix $\mathbf{X}$ containing $m = 2,429$ gray-scale face images with $n = 19 \times 19 = 361$ pixels per face. Each image (row) is scaled to have mean and standard deviation 0.25.  
Read in the [`nnmf-2429-by-361-face.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt) file, e.g., using [`readdlm()`](https://docs.julialang.org/en/stable/stdlib/io-network/#Base.DataFmt.readdlm-Tuple{Any,Char,Type,Char}) function, and display a couple sample images, e.g., using [ImageView.jl](https://github.com/JuliaImages/ImageView.jl) package.

0. Report the run times, using `@time`, of your function for fitting NNMF on the MIT CBCL face data set at ranks $r=10, 20, 30, 40, 50$. For ease of comparison (and grading), please start your algorithm with the provided $\mathbf{V}^{(0)}$ (first $r$ columns of [`V0.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/V0.txt)) and $\mathbf{W}^{(0)}$ (first $r$ rows of [`W0.txt`](http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/W0.txt)) and stopping criterion
$$
	\frac{|L^{(t+1)} - L^{(t)}|}{|L^{(t)}| + 1} \le 10^{-4}.
$$

0. Choose an $r \in \{10, 20, 30, 40, 50\}$ and start your algorithm from a different $\mathbf{V}^{(0)}$ and $\mathbf{W}^{(0)}$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

0. For the same $r$, start your algorithm from $v_{ik}^{(0)} = w_{kj}^{(0)} = 1$ for all $i,j,k$. Do you obtain the same objective value and $(\mathbf{V}, \mathbf{W})$? Explain what you find.

0. Plot the basis images (rows of $\mathbf{W}$) at rank $r=50$. What do you find?

0. (Optional) Investigate the GPU capabilities of Julia. Report the speed gain of your GPU code over CPU code at ranks $r=10, 20, 30, 40, 50$. Make sure to use the same starting point as in part 2.

In [13]:
# possible for efficiency: column major/ use blas or lapack/overwrite the initial V and W
function nnmf(
    X::Matrix{T}, 
    r::Int;
    maxiter::Int=1000, 
    tol::T=1e-4,
    V::Matrix{T}=rand(T, size(X, 1), r),
    W::Matrix{T}=rand(T, r, size(X, 2))
    ) where T <: AbstractFloat
    # implementation
    m = size(X,1)
    n = size(X,2)
    for iter = 1:maxiter
        #b matrix
        b = *(V, W)
        #to update matrix V
        for i = 1:m
            for k = 1:r
                V[i, k] = V[i, k] * dot(X[i,:], W[k,:])/dot(b[i,:], W[k,:])
            end
        end
        
        #to update matrix W
        #update matrix b
        b = *(V, W)
        for k = 1:r
            for j = 1:n
                W[k, j] = W[k, j] * dot(X[:,j], V[:,k])/dot(b[:,j], V[:,k])
            end
        end      
        # if reaches maxiter (and not satisfy the tolerance, stop with error message)
        #@show f_normdiff_square = (vecnorm(X - *(V, W)))^2
        f_normdiff_square = (vecnorm(X - *(V, W)))^2
        if f_normdiff_square < tol
            break
        end
    end
    # Output
    return V, W
end

nnmf (generic function with 1 method)

In [6]:
# possible for efficiency: column major/ use blas or lapack/overwrite the initial V and W
function nnmf2(
    X::Matrix{T}, 
    r::Int;
    maxiter::Int=1000, 
    tol::T=1e-4,
    V::Matrix{T}=rand(T, size(X, 1), r),
    W::Matrix{T}=rand(T, r, size(X, 2))
    ) where T <: AbstractFloat
    # implementation
    m = size(X,1)
    n = size(X,2)
    for iter = 1:maxiter
        #B matrix
        B = *(V, W)
        #to update matrix V
        V = V .* (X * W.') ./ (B * W.')
      
        #to update matrix W, first update matrix b
        B = *(V, W)
        
        W = W .* (V.' * X) ./ (V.' * B)
             
        # if reaches maxiter (and not satisfy the tolerance, stop with error message)
        #@show f_normdiff_square = (vecnorm(X - *(V, W)))^2
        f_normdiff_square = (vecnorm(X - *(V, W)))^2
        if f_normdiff_square < tol
            break
        end
    end
    # Output
    return V, W
end

nnmf2 (generic function with 1 method)

x = rand(2, 2) = [0.290547 0.692282; 0.604706 0.0225193]
y = rand(2, 2) = [0.322799 0.896093; 0.641479 0.796329]
x.' = [0.290547 0.604706; 0.692282 0.0225193]


2×2 Array{Float64,2}:
 0.290547  0.604706 
 0.692282  0.0225193

In [11]:
x = [3 3 4 5 2.0; 4 5 2 1 5; 52 23 2 5 1; 42 3 3 5 1]
t = nnmf(x,4);

V = [0.00271992 0.697963 0.709873 0.26567; 0.32284 0.84455 0.336996 0.763535; 0.887431 0.394595 0.0375297 0.888648; 0.150248 0.074771 0.664579 0.972106]


In [10]:
matrixt = readdlm("nnmf-2429-by-361-face.txt");
#nnmf(matrixt, 10)

In [22]:
#@time nnmf(matrixt,10)
using BenchmarkTools
t = matrixt[1:3,1:3];
 
@show nnmf2(t,2,V = [1 1; 1 1.0;1 1],W = [1 1 1.0; 1 1.0 1])
@show nnmf(t,2,V = [1 1; 1 1.0;1 1],W = [1 1 1.0; 1 1.0 1])
#@benchmark nnmf(t,10)
#size(t)

nnmf2(t, 2, V=[1 1; 1 1.0; 1 1], W=[1 1 1.0; 1 1.0 1]) = ([0.0365886 0.0365886; 0.0555727 0.0555727; 0.00918324 0.00918324], [1.89489 0.664596 0.26329; 1.89489 0.664596 0.26329])
nnmf(t, 2, V=[1 1; 1 1.0; 1 1], W=[1 1 1.0; 1 1.0 1]) = ([0.0365886 0.0365886; 0.0555727 0.0555727; 0.00918324 0.00918324], [1.89489 0.664596 0.26329; 1.89489 0.664596 0.26329])


([0.0365886 0.0365886; 0.0555727 0.0555727; 0.00918324 0.00918324], [1.89489 0.664596 0.26329; 1.89489 0.664596 0.26329])

In [6]:
#Pkg.add("CUDAnative")
#Pkg.add("CUDAdrv")

#using CUDAnative
Pkg.build("CUDAdrv")

INFO: Building CUDAdrv
===============================[ ERROR: CUDAdrv ]===============================

LoadError: MethodError: no method matching dlopen(::Void)
Closest candidates are:
  dlopen(::Symbol) at libdl.jl:94
  dlopen(::Symbol, ::Integer) at libdl.jl:94
  dlopen(::AbstractString) at libdl.jl:97
  ...
while loading /Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6/CUDAdrv/deps/build.jl, in expression starting on line 104


================================[ BUILD ERRORS ]================================


 - packages with build errors remain installed in /Applications/JuliaPro-0.6.2.2.app/Contents/Resources/pkgs-0.6.2.2/v0.6
 - build the package(s) and all dependencies with `Pkg.build("CUDAdrv")`
 - build a single package by running its `deps/build.jl` script



In [ ]:
readdlm(download("http://hua-zhou.github.io/teaching/biostatm280-2018spring/hw/hw2/nnmf-2429-by-361-face.txt"))

## Q2. Linear Mixed Models

Consider a linear mixed effects model
$$
	y_i = \mathbf{x}_i^T \beta + \mathbf{z}_i^T \gamma + \epsilon_i, \quad i=1,\ldots,n,
$$
where $\epsilon_i$ are independent normal errors $N(0,\sigma_0^2)$, $\beta \in \mathbb{R}^p$ are fixed effects, and $\gamma \in \mathbb{R}^q$ are random effects assumed to be $N(\mathbf{0}_q, \sigma_1^2 \mathbf{I}_q$) independent of $\epsilon_i$. 

0. Show that 
$$
    \mathbf{y} \sim N \left( \mathbf{X} \beta, \sigma_0^2 \mathbf{I}_n + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T \right),
$$
where $\mathbf{y} = (y_1, \ldots, y_n)^T \in \mathbb{R}^n$, $\mathbf{X} = (\mathbf{x}_1, \ldots, \mathbf{x}_n)^T \in \mathbb{R}^{n \times p}$, and $\mathbf{Z} = (\mathbf{z}_1, \ldots, \mathbf{z}_n)^T \in \mathbb{R}^{n \times q}$. 

0. Write a function, with interface 
    ```julia
    logpdf_mvn(y::Vector, Z::Matrix, σ0::Number, σ1::Number),
    ```
that evaluates the log-density of a multivariate normal with mean $\mathbf{0}$ and covariance $\sigma_0^2 \mathbf{I} + \sigma_1^2 \mathbf{Z} \mathbf{Z}^T$ at $\mathbf{y}$. Make your code efficient in the $n \gg q$ case. 

0. Compare your result (both accuracy and timing) to the [Distributions.jl](http://distributionsjl.readthedocs.io/en/latest/multivariate.html#multivariate-normal-distribution) package using following data.  
    ```julia
    using BenchmarkTools, Distributions

    srand(280)
    n, q = 2000, 10
    Z = randn(n, q)
    σ0, σ1 = 0.5, 2.0
    Σ = σ1^2 * Z * Z.' + σ0^2 * I
    mvn = MvNormal(Σ) # MVN(0, Σ)
    y = rand(mvn) # generate one instance from MNV(0, Σ)

    # check you answer matches that from Distributions.jl
    @show logpdf_mvn(y, Z, σ0, σ1)
    @show logpdf(mvn, y)

    # benchmark
    @benchmark logpdf_mvn(y, Z, σ0, σ1)
    @benchmark logpdf(mvn, y)
    ```